In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

In [3]:
od.download('https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aryan7004
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench


100%|██████████| 3.51M/3.51M [00:00<00:00, 5.65MB/s]


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [5]:
from nltk.corpus import stopwords
import re

In [6]:
from tensorflow import keras
from keras import layers

In [7]:
df = pd.read_csv('/content/language-translation-englishfrench/eng_-french.csv')

In [8]:
df.head()

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175621 entries, 0 to 175620
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   English words/sentences  175621 non-null  object
 1   French words/sentences   175621 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB


In [10]:
df['English words/sentences'].drop_duplicates(inplace = True)
df['French words/sentences'].drop_duplicates(inplace = True)

In [11]:
df['English words/sentences'].nunique()

123100

In [12]:
df['French words/sentences'].nunique()

165975

In [13]:
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [14]:
# text preprocessing
def preprocess(text,language):
  text = text.lower()
  if language=='English':
    text = ' '.join(contractions[word] if word in contractions else word for word in text.split())
  text = re.sub(r"[.'!#$%&\'()*+,-./:;<=>?@[\\]^ `{|}~]"," ",text)
  text = ' '.join([word for word in text.split()])
  return text

In [15]:
df["English words/sentences"] = df['English words/sentences'].apply(lambda x:preprocess(x,"English"))
df["French words/sentences"] = df['French words/sentences'].apply(lambda x:preprocess(x,"French"))

In [16]:
df.head()

,English words/sentences,French words/sentences
0,hi.,salut!
1,run!,cours !
2,run!,courez !
3,who?,qui ?
4,wow!,ça alors !


In [17]:
sentence_en = df['English words/sentences'].to_numpy()
sentence_fr = df['French words/sentences'].to_numpy()
valid_fraction = 0.1
valid_len = int(len(df)*valid_fraction)
train_en = sentence_en[:-valid_len]
train_fr = sentence_fr[:-valid_len]
valid_en = sentence_en[-valid_len:]
valid_fr = sentence_fr[-valid_len:]

In [26]:
def prepare_input_and_target_dataset(sentence_en,sentence_fr):
  return (sentence_en,b"startofseq "+sentence_fr),sentence_fr+b" endofseq"

In [ ]:
def datasetfrom_sentence(sentence_en,
                         sentence_fr,
                         batch_size = 32,
                         cache = True,
                         shuffle = False,
                         shuffle_buffer_size = 10_000,
                         seed = None):
  dataset = tf.data.Dataset.from_tensor_slices((sentence_en,sentence_fr))
  dataset = dataset.map(prepare_input_and_target_dataset,num_parallel_calls = tf.data.AUTOTUNE)
  if cache:
    dataset = dataset.cache()
  if shuffle:
    dataset = dataset.shuffle(shuffle_buffer_size)
  return dataset.batch(batch_size)


In [ ]:
train_ds = datasetfrom_sentence(train_en,train_fr)

In [ ]:
print(train_ds)

<_BatchDataset element_spec=((TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.string, name=None)), TensorSpec(shape=(None,), dtype=tf.string, name=None))>


In [ ]:
valid_ds = datasetfrom_sentence(valid_en,valid_fr)

In [ ]:
list(train_ds.take(1))[0]

((<tf.Tensor: shape=(32,), dtype=string, numpy=
  array([b'hi.', b'run!', b'run!', b'who?', b'wow!', b'fire!', b'help!',
         b'jump.', b'stop!', b'stop!', b'stop!', b'wait!', b'wait!',
         b'go on.', b'go on.', b'go on.', b'hello!', b'hello!', b'i see.',
         b'i try.', b'i won!', b'i won!', b'i won.', b'oh no!', b'attack!',
         b'attack!', b'cheers!', b'cheers!', b'cheers!', b'cheers!',
         b'get up.', b'go now.'], dtype=object)>,
  <tf.Tensor: shape=(32,), dtype=string, numpy=
  array([b'startofseq salut!', b'startofseq cours !',
         b'startofseq courez !', b'startofseq qui ?',
         b'startofseq \xc3\xa7a alors !', b'startofseq au feu !',
         b"startofseq \xc3\xa0 l'aide !", b'startofseq saute.',
         b'startofseq \xc3\xa7a suffit !', b'startofseq stop !',
         b'startofseq arr\xc3\xaate-toi !', b'startofseq attends !',
         b'startofseq attendez !', b'startofseq poursuis.',
         b'startofseq continuez.', b'startofseq poursuivez.'

In [ ]:
from keras import Sequential
from keras.layers import Embedding,Dense,LSTM, Bidirectional, Attention,TextVectorization
class BidirectionalEncoderandDecoderWithAttention(keras.Model):
  def __init__(
      self,
      vocabulary_size = 5000,
      max_length_size=50,
      embedding_size = 256,
      units_lstm =512,
      **kwargs):
    super().__init__(**kwargs)
    self.max_sentence_len = max_length_size
    self.vocabulary_size = vocabulary_size

    # tokenizer for english
    self.tokenizer_en = TextVectorization(vocabulary_size,output_sequence_length = max_length_size)
    self.tokenizer_fr = TextVectorization(vocabulary_size,output_sequence_length = max_length_size)
    self.encoder_embeddings = Embedding(vocabulary_size,embedding_size,mask_zero = True)
    self.encoder = Bidirectional(LSTM(units_lstm//2,return_sequences = True,return_state = True))
    self.decoder_embeddings = Embedding(vocabulary_size,embedding_size,mask_zero = True)
    self.decoder = LSTM(units_lstm,return_sequences = True)
    self.attention = Attention()
    self.output_layer = Dense(vocabulary_size,activation = 'softmax')
  def call(self,inputs):
      encoder_inputs,decoder_inputs = inputs

      encoder_input_ids = self.tokenizer_en(encoder_inputs)

      decoder_input_ids = self.tokenizer_fr(decoder_inputs)

      encoder_embeddings = self.encoder_embeddings(encoder_input_ids)
      decoder_embeddings = self.decoder_embeddings(decoder_input_ids)

      encoder_op,*encoder_state = self.encoder(encoder_embeddings)
      encoder_state = [
            tf.concat(encoder_state[0::2], axis=-1),
            tf.concat(encoder_state[1::2], axis=-1),
        ]
      decoder_op = self.decoder(decoder_embeddings,initial_state = encoder_state)
      attention_output = self.attention([decoder_op,encoder_op])
      output = self.output_layer(attention_output)
      return output

In [ ]:
def adapts_compile_fit(model,
                       train_data,
                       valid_data,
                       n_epochs = 40,
                       n_patience = 20,
                       init_lr = 0.001,
                       lr_decay_rate = 0.1):
  model.tokenizer_en.adapt(train_data.map(lambda sentence,target:sentence[0],num_parallel_calls = tf.data.AUTOTUNE,))
  model.tokenizer_fr.adapt(train_data.map(lambda sentence,target:sentence[1]+b" endofseq",num_parallel_calls = tf.data.AUTOTUNE))

  train_data_prepared = train_data.map(
        lambda sentences, target: (sentences,model.tokenizer_fr(target)),
        num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

  valid_data_prepared = valid_data.map(
        lambda sentences, target: (sentences,model.tokenizer_fr(target)),
        num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

  early_stopping = keras.callbacks.EarlyStopping(monitor="val_accuracy",patience = n_patience,restore_best_weights = True)
  n_decay_steps = n_epochs * len(list(train_data_prepared))
  scheduled_lr = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=init_lr,
        decay_steps=n_decay_steps,
        decay_rate=lr_decay_rate)
  model.compile(loss='sparse_categorical_crossentropy',
                optimizer = keras.optimizers.RMSprop(learning_rate = scheduled_lr),
                metrics = ['accuracy'])
  return model.fit(train_data_prepared,
                   epochs = n_epochs,
                   validation_data = valid_data_prepared,
                   callbacks = [early_stopping]
                   ,verbose=1)


In [ ]:
def prepare_input_and_target_dataset(sentence_en,sentence_fr):
  return (sentence_en,b"startofseq "+sentence_fr),sentence_fr+b" endofseq"

def datasetfrom_sentence(sentence_en,
                         sentence_fr,
                         batch_size = 32,
                         cache = True,
                         shuffle = False,
                         shuffle_buffer_size = 10_000,
                         seed = None):
  dataset = tf.data.Dataset.from_tensor_slices((sentence_en,sentence_fr))
  dataset = dataset.map(prepare_input_and_target_dataset,num_parallel_calls = tf.data.AUTOTUNE)
  if cache:
    dataset = dataset.cache()
  if shuffle:
    dataset = dataset.shuffle(shuffle_buffer_size)
  return dataset.batch(batch_size)

def adapts_compile_fit(model,
                       train_data,
                       valid_data,
                       n_epochs = 40,
                       n_patience = 20,
                       init_lr = 0.001,
                       lr_decay_rate = 0.1):
  model.tokenizer_en.adapt(train_data.map(lambda sentence,target:sentence[0],num_parallel_calls = tf.data.AUTOTUNE,))
  model.tokenizer_fr.adapt(train_data.map(lambda sentence,target:sentence[1]+b" endofseq",num_parallel_calls = tf.data.AUTOTUNE))

  train_data_prepared = train_data.map(
        lambda sentences, target: (sentences,model.tokenizer_fr(target)),
        num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

  valid_data_prepared = valid_data.map(
        lambda sentences, target: (sentences,model.tokenizer_fr(target)),
        num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

  early_stopping = keras.callbacks.EarlyStopping(monitor="val_accuracy",patience = n_patience,restore_best_weights = True)
  n_decay_steps = n_epochs * len(list(train_data_prepared))
  scheduled_lr = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=init_lr,
        decay_steps=n_decay_steps,
        decay_rate=lr_decay_rate)
  model.compile(loss='sparse_categorical_crossentropy',
                optimizer = keras.optimizers.RMSprop(learning_rate = scheduled_lr),
                metrics = ['accuracy'])
  return model.fit(train_data_prepared,
                   epochs = n_epochs,
                   validation_data = valid_data_prepared,
                   callbacks = [early_stopping]
                   ,verbose=1)


In [ ]:
model = BidirectionalEncoderandDecoderWithAttention(max_length_size=15)

In [ ]:
history = adapts_compile_fit(model,train_ds,valid_ds,init_lr = 0.01,lr_decay_rate=0.01)

Epoch 1/40
4940/4940 [==============================] - 144s 26ms/step - loss: 2.9255 - accuracy: 0.4801 - val_loss: 2.7329 - val_accuracy: 0.4586
Epoch 2/40
4940/4940 [==============================] - 103s 21ms/step - loss: 1.9362 - accuracy: 0.5931 - val_loss: 2.5315 - val_accuracy: 0.4867
Epoch 3/40
4940/4940 [==============================] - 103s 21ms/step - loss: 1.6596 - accuracy: 0.6340 - val_loss: 2.4653 - val_accuracy: 0.5006
Epoch 4/40
4940/4940 [==============================] - 104s 21ms/step - loss: 1.4915 - accuracy: 0.6604 - val_loss: 2.4133 - val_accuracy: 0.5129
Epoch 5/40
4940/4940 [==============================] - 104s 21ms/step - loss: 1.3552 - accuracy: 0.6822 - val_loss: 2.4107 - val_accuracy: 0.5185
Epoch 6/40
4940/4940 [==============================] - 112s 23ms/step - loss: 1.2375 - accuracy: 0.7019 - val_loss: 2.4100 - val_accuracy: 0.5203
Epoch 7/40
4940/4940 [==============================] - 111s 22ms/step - loss: 1.1278 - accuracy: 0.7211 - val_loss: 2

In [ ]:
def translate(model,sentence_en):
  answer = ""
  for i in range(model.max_sentence_len):
    X_encoded = np.array([sentence_en])
    X_decoded = np.array(["startofseq "+answer])
    y_prob = model.predict((X_encoded,X_decoded),verbose=0)[0,i]
    y_prob_id = np.argmax(y_prob)
    predicted_word = model.tokenizer_fr.get_vocabulary()[y_prob_id]
    if predicted_word==" endofseq":
      break
    answer = answer + " " + predicted_word
  return answer.strip()

In [ ]:
translate_1 = translate(model,"wow how?")
print(translate_1)

In [ ]:
class PositionalEncoding(layers.Layer):

  def __init__(
      self,max_sentence_len = 50,embedding_size = 256, dtype = tf.float32, **kwargs
      ):
    super().__init__(dtype = dtype,**kwargs)
    if embedding_size %2!=0:
      raise ValueError("embedding_size must be even")
    p,i = np.meshgrid(np.arange(max_sentence_len),np.arange(embedding_size//2))
    pos_emb = np.empty((1,max_sentence_len,embedding_size))
    pos_emb[:,:,0::2] = np.sin(p/10_000**(2*i/embedding_size)).T
    pos_emb[:,:,1::2] = np.cos(p/10_000**(2*i/embedding_size)).T
    self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))
    self.supports_masking = True

  def call(self, inputs):
    batch_max_length = tf.shape(inputs)[1]
    return inputs + self.positional_embedding[:, :batch_max_length]


In [ ]:
class Encoder(layers.Layer):

  def __init__(
      self,embedding_size = 256, n_attention_heads = 8,n_units_dense = 256, dropout_rate = 0.2,**kwargs
      ):
    super().__init__(**kwargs)
    self.multi_head_attention = layers.MultiHeadAttention(n_attention_heads, embedding_size, dropout = dropout_rate)
    self.feed_forward = keras.Sequential(
        [
            layers.Dense(n_units_dense, activation = "relu", kernel_initializer = "he_normal"),
            layers.Dense(embedding_size, kernel_initializer= "he_normal")
        ]
    )
    self.add = layers.Add()
    self.normalization = layers.LayerNormalization()

  def call(self,inputs,mask=None):
      Z= inputs
      skip_z = Z
      Z = self.multi_head_attention(Z,value = Z,attention_mask = mask)
      Z = self.normalization(self.add([Z,skip_z]))
      skip_z = Z

      Z = self.feed_forward(Z)
      Z = self.normalization(self.add([Z,skip_z]))
      return Z



In [ ]:
class Decoder(layers.Layer):

  def __init__(
      self,embedding_size = 256,n_attention_head = 8,n_units_dense = 256,dropout_rate = 0.2,**kwargs
      ):
    super().__init__(**kwargs)
    self.masked_multi_head_attention = layers.MultiHeadAttention(n_attention_head,embedding_size,dropout = dropout_rate)
    self.multi_head_attention = layers.MultiHeadAttention(n_attention_head, embedding_size, dropout = dropout_rate)
    self.normalization = layers.Normalization()
    self.add = layers.Add()
    self.feed_forward = keras.Sequential(
        [
            layers.Dense(n_units_dense,activation = "relu",kernel_initializer = "he_normal"),
            layers.Dense(embedding_size,kernel_initializer = "he_normal"),
            layers.Dropout(dropout_rate),
        ]
        )
  def call(self,inputs,mask=None):
    decoder_mask, encoder_mask = mask
    Z, encoder_output = inputs
    Z_skip = Z
    Z = self.masked_multi_head_attention(Z,value=Z,attention_mask = decoder_mask)
    Z = self.normalization(self.add([Z_skip,Z]))
    Z_skip = Z
    Z = self.multi_head_attention(Z,value = encoder_output,attention_mask = encoder_mask)
    Z = self.normalization(self.add([Z,Z_skip]))
    Z_skip = Z
    Z = self.feed_forward(Z)
    Z = self.normalization(self.add([Z_skip,Z]))
    return Z

In [ ]:
class Transformer(keras.Model):

  def __init__(
      self,
      vocabulary_size = 5000,
      max_sentence_len = 50,
      embedding_size = 256,
      n_encoder_decoder_blocks = 1,
      n_attention_heads = 8,
      n_units_dense = 256,
      dropout_rate = 0.2,
      **kwargs
  ):
   super().__init__(**kwargs)
   self.max_sentence_len = max_sentence_len
   self.tokenizer_en = layers.TextVectorization(
      vocabulary_size, output_sequence_length = max_sentence_len
   )
   self.tokenizer_fr = layers.TextVectorization(
      vocabulary_size, output_sequence_length = max_sentence_len
   )
   self.encoder_embedding = layers.Embedding(
      vocabulary_size,embedding_size, mask_zero = True
   )
   self.decoder_embedding = layers.Embedding(
      vocabulary_size, embedding_size, mask_zero = True
   )
   self.positional_encoding = PositionalEncoding(max_sentence_len,embedding_size)
   self.encoder_blocks = [
      Encoder(embedding_size,n_attention_heads,n_units_dense,dropout_rate)
      for _ in range(n_encoder_decoder_blocks)
   ]
   self.decoder_blocks = [
      Decoder(embedding_size,n_attention_heads,n_units_dense,dropout_rate)
      for _ in range(n_encoder_decoder_blocks)
   ]
   self.output_layer = layers.Dense(vocabulary_size,activation = 'softmax')

  def call(self,inputs):
   encoder_inputs, decoder_inputs = inputs

   encoder_input_ids = self.tokenizer_en(encoder_inputs)
   decoder_input_ids = self.tokenizer_fr(decoder_inputs)

   encoder_embeddings = self.encoder_embedding(encoder_input_ids)
   decoder_embeddings = self.decoder_embedding(decoder_input_ids)

   encoder_pos_embeddings = self.positional_encoding(encoder_embeddings)
   decoder_pos_embeddings = self.positional_encoding(decoder_embeddings)

   encoder_pad_mask = tf.math.not_equal(encoder_input_ids,0)[:,tf.newaxis]
   decoder_pad_mask = tf.math.not_equal(decoder_input_ids,0)[:,tf.newaxis]
   batch_max_len_decoder = tf.shape(decoder_embeddings)[1]
   decoder_casual_mask = tf.linalg.band_part(tf.ones((batch_max_len_decoder, batch_max_len_decoder),tf.bool),-1,0)

   decoder_mask = decoder_casual_mask & decoder_pad_mask

   Z = encoder_pos_embeddings
   for encoder_block in self.encoder_blocks:
     Z = encoder_block(Z,mask = encoder_pad_mask)
   encoder_output = Z
   Z = decoder_pos_embeddings
   for decoder_block in self.decoder_blocks:
     Z = decoder_block(
        [Z, encoder_output], mask = [decoder_mask, encoder_pad_mask]
    )
   Z = self.output_layer(Z)
   return Z

In [ ]:
transformer_model = Transformer(max_sentence_len=15)

In [ ]:
history_2 = adapts_compile_fit(transformer_model,train_ds,valid_ds,n_epochs=100,init_lr = 0.003,lr_decay_rate=0.04)

Epoch 1/100
4940/4940 [==============================] - 143s 27ms/step - loss: 2.4278 - accuracy: 0.6553 - val_loss: 3.9308 - val_accuracy: 0.3368
Epoch 2/100
4940/4940 [==============================] - 117s 24ms/step - loss: 1.8647 - accuracy: 0.6777 - val_loss: 3.9609 - val_accuracy: 0.3362
Epoch 3/100
4940/4940 [==============================] - 117s 24ms/step - loss: 1.8389 - accuracy: 0.6794 - val_loss: 3.7836 - val_accuracy: 0.3431
Epoch 4/100
4940/4940 [==============================] - 118s 24ms/step - loss: 1.8217 - accuracy: 0.6804 - val_loss: 3.7841 - val_accuracy: 0.3407
Epoch 5/100
4940/4940 [==============================] - 118s 24ms/step - loss: 1.8118 - accuracy: 0.6811 - val_loss: 3.7925 - val_accuracy: 0.3440
Epoch 6/100
4940/4940 [==============================] - 113s 23ms/step - loss: 1.8064 - accuracy: 0.6813 - val_loss: 3.7753 - val_accuracy: 0.3483
Epoch 7/100
 737/4940 [===>..........................] - ETA: 1:31 - loss: 1.1199 - accuracy: 0.8075

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import time
import spacy
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torchtext
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
import multiprocessing

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
!python -m spacy download fr

⚠ As of spaCy v3.0, shortcuts like 'fr' are deprecated. Please use the
full pipeline package name 'fr_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 42.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [30]:
en_tokenizer = spacy.load("en_core_web_sm")
fr_tokenizer = spacy.load("fr_core_news_sm")

In [31]:
def data_process(sentence_en,sentence_fr):
  data = []
  for (en_sen,fr_sen) in zip(sentence_en,sentence_fr):
    data.append((en_sen,fr_sen))
  return data
def generate_batch(data_batch):
  en_batch,fr_batch = [],[]
  for (en_item,fr_item) in data_batch:
    en_batch.append(['<bos>'+en_item + '<eos>'])
    fr_batch.append(['<bos>'+fr_item + '<eos>'])
  return en_batch,fr_batch

In [32]:
train_data = data_process(train_en,train_fr)
valid_data = data_process(valid_en,valid_fr)

In [33]:
train_iter = DataLoader(train_data, batch_size=32,
                        shuffle=True, collate_fn=generate_batch)

In [34]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

def yield_tokens_en(data_iter):
    for (en_batch, fr_batch) in data_iter:
        for en_sent in en_batch:
            my_string = ' '.join(en_sent)
            en_tokens = my_string.split()  # Split English sentence into tokens
            yield en_tokens

def yield_tokens_fr(data_iter):
    for (en_batch, fr_batch) in data_iter:
        for fr_sent in fr_batch:
            my_string = ' '.join(fr_sent)
            fr_tokens = my_string.split()  # Split English sentence into tokens
            yield fr_tokens
en_vocab = build_vocab_from_iterator(yield_tokens_en(train_iter),
                                  min_freq=1,
                                  specials=special_symbols,
                                  special_first=True)
fr_vocab = build_vocab_from_iterator(yield_tokens_fr(train_iter),
                                  min_freq=1,
                                  specials=special_symbols,
                                  special_first=True)
en_vocab.set_default_index(en_vocab['<unk>'])
fr_vocab.set_default_index(fr_vocab['<unk>'])

In [35]:
def collate_fn(data_batch):
  en_batch = []
  fr_batch = []
  for (en_item,fr_item) in data_batch:
    en_ids = torch.tensor([en_vocab[token] for token in en_item],dtype = torch.long)
    fr_ids = torch.tensor([fr_vocab[token] for token in fr_item],dtype = torch.long)
    # print(type(en_ids))
    en_batch.append(torch.cat([torch.tensor([en_vocab['<bos>']]), en_ids ,torch.tensor([en_vocab['<eos>']])], dim=0))
    fr_batch.append(torch.cat([torch.tensor([fr_vocab['<bos>']]), fr_ids ,torch.tensor([fr_vocab['<eos>']])], dim=0))
    # print(type(e_tensor))
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  fr_batch = pad_sequence(fr_batch, padding_value=PAD_IDX)
  return en_batch,fr_batch

In [36]:
train_data_token_iter = DataLoader(train_data, batch_size=32,
                        shuffle=True, collate_fn=collate_fn)
valid_data_token_iter = DataLoader(valid_data, batch_size=32,
                        shuffle=True, collate_fn=collate_fn)

In [47]:
class PositionalEmbedding(nn.Module):
    def __init__(self,
                 emb_size=512,
                 maxlen = 50):
        super(PositionalEmbedding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        return token_embedding + self.pos_embedding[:token_embedding.size(0), :]

In [46]:
class MultiHeadAttention(nn.Module):
  def __init__(self,embed_dim=512,n_heads = 8):
    super(MultiHeadAttention,self).__init__()
    self.embed_dim = embed_dim
    self.n_heads = n_heads
    self.single_head_dim = int(self.embed_dim/self.n_heads)

    self.query_matrix = nn.Linear(self.single_head_dim,self.single_head_dim,bias = False)
    self.key_matrix = nn.Linear(self.single_head_dim,self.single_head_dim,bias = False)
    self.value_matrix = nn.Linear(self.single_head_dim,self.single_head_dim,bias = False)
    self.output = nn.Linear(self.single_head_dim*self.n_heads,self.embed_dim)

  def forward(self,key,query,value,mask=None):
    batch_size = key.size(0)
    seq_length = key.size(1)
    seq_query_len = query.size(1)
    key = key.view(batch_size,seq_length,self.n_heads,self.single_head_dim)
    query = query.view(batch_size,seq_query_len,self.n_heads,self.single_head_dim)
    value = value.view(batch_size,seq_length,self.n_heads,self.single_head_dim)

    k = self.key_matrix(key)
    q = self.query_matrix(query)
    v = self.value_matrix(value)

    k = k.transpose(1,2)
    q = q.transpose(1,2)
    v = v.transpose(1,2)

    k_adjusted = k.transpose(-1,-2)

    product = torch.matmul(q,k_adjusted)

    if mask is not None:
      product = product.masked_fill(mask==0,float("-1e20"))

    product = product/math.sqrt(self.single_head_dim)
    scores = F.softmax(product,dim=-1)
    scores = torch.matmul(scores,v)
    concat = scores.transpose(1,2).contiguous().view(batch_size,seq_query_len,self.single_head_dim*self.n_heads)
    output = self.output(concat)

In [45]:
class FeedForwardNetwork(nn.Module):
  def __init__(self,embed_dim,expansion_factor=4):
    super(FeedForwardNetwork,self).__init__()
    self.fc1 = nn.Linear(embed_dim,expansion_factor*embed_dim)
    self.fc2 = nn.Linear(expansion_factor*embed_dim,embed_dim)
    self.relu = nn.ReLU()

  def forward(self,x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

In [44]:
class EncoderBlock(nn.Module):
  def __init__(self,embed_dim,n_heads=8,expansion_factor=4):
    super(EncoderBlock,self).__init__()
    self.attention = MultiHeadAttention(embed_dim,n_heads)
    self.feed_forward = FeedForwardNetwork(embed_dim,expansion_factor*embed_dim)
    self.norm_1 = nn.LayerNorm(embed_dim)
    self.norm_2 = nn.LayerNorm(embed_dim)
    self.dropout = nn.Dropout(0.2)

  def forward(self,x,mask):
    attention_output = self.attention(x,x,x,mask)
    x = self.norm_1(x + self.dropout(attention_output))
    ff_output = self.feed_forward(x)
    x = self.norm_2(x + self.dropout(ff_output))
    return x

In [43]:
class DecoderBlock(nn.Module):
  def __init__(self,embed_dim,n_heads=8,expansion_factor=4):
    super(DecoderBlock,self).__init__()
    self.masked_attention = MultiHeadAttention(embed_dim,n_heads)
    self.norm1 = nn.LayerNorm(embed_dim)
    self.attention = MultiHeadAttention(embed_dim,n_heads)
    self.norm2 = nn.LayerNorm(embed_dim)
    self.feed_forward = FeedForwardNetwork(embed_dim,expansion_factor*embed_dim)
    self.norm3 = nn.LayerNorm(embed_dim)
    self.dropout = nn.Dropout(0.2)

  def forward(self,x,encoder_output,source_mask,target_mask):
    masked_output = self.masked_output(x,x,x,target_mask)
    x = self.norm1(x + self.dropout(masked_output))
    attention_output = self.attention(encoder_output,encoder_output,x,source_mask)
    x = self.norm2(x + self.dropout(attention_output))
    ff_output = self.feed_forward(x)
    x = self.norm3(x + self.dropout(ff_output))
    return x


In [41]:
class Transformer(nn.Module):
  def __init__(self,input_vocab_size,output_vocab_size,embed_dim=512,n_heads=8,expansion_factor=4,max_sentence_len=50,num_layers=2):
    super(Transformer,self).__init__()
    self.max_len = max_sentence_len
    # self.tokenizer_en = en_tokenizer
    # self.tokenizer_fr = fr_tokenizer
    self.input_embedding = nn.Embedding(input_vocab_size,embed_dim)
    self.positional_encoding = PositionalEmbedding(max_sentence_len,embed_dim)
    self.output_embedding = nn.Embedding(output_vocab_size,embed_dim)
    self.encoder_layer = nn.ModuleList([EncoderBlock(embed_dim,n_heads,expansion_factor)])
    self.decoder_layer = nn.ModuleList([DecoderBlock(embed_dim,n_heads,expansion_factor)])
    self.fc = nn.Linear(embed_dim,output_vocab_size)
    self.dropout = nn.Dropout(0.2)
  def generate_mask(self,source,target):
    source_mask = (source!=0).unsqueeze(1).unsqueeze(2)
    target_mask = (target!=0).unsqueeze(2).unsqueeze(3)
    seq_length = target.size(1)
    nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
    target_mask = target_mask&nopeak_mask
    return source_mask,target_mask
  def forward(self,source,target):
    source_mask,target_mask = self.generate_mask(source,target)
    # source_ids = [en_vocab[token] for token in self.tokenizer_en(source)]
    # source_padded_ids = F.pad(source_ids,pad=(0, self.max_len - len(source_ids)),mode = 'constant',value = en_vocab['<pad>'])
    # source_padded_ids = torch.tensor(source_padded_ids,dtype = torch.long)
    # print(type(source_padded_ids))
    ip_embed = self.input_embedding(source)
    ip_positional_encoding = self.positional_encoding(ip_embed)
    ip_embedded = self.dropout(ip_positional_encoding)
    encoder_output = ip_embedded
    for enc_layer in self.encoder_layer:
      encoder_output = enc_layer(encoder_output,source_mask)
    # target_ids = [fr_vocab[token] for token in self.tokenizer_fr(target)]
    # target_padded_ids = F.pad(target_ids,pad=(0, self.max_len - len(target_ids)),mode = 'constant',value = en_vocab['<pad>'])
    # target_padded_ids = torch.tensor(target_padded_ids,dtype = torch.long)
    # print(type(target_padded_ids))
    op_embed = self.output_embedding(target)
    op_positional_encoding = self.positional_encoding(op_embed)
    op_embedded = self.dropout(op_positional_encoding)
    for dec_layer in self.decoder_layer:
      decoder_output = dec_layer(decoder_output,source_mask,target_mask)
    output = self.fc(decoder_output)
    return output

In [38]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [39]:
def train(model,data_iterator,optimizer,criterion,clip):

  model.train()

  epoch_loss = 0
  epoch_total = 0
  epoch_corrects = 0

  for _,(source,target) in enumerate(data_iterator):
    print(source)
    print(type(source))
    source,target = source.to(device),target.to(device)

    optimizer.zero_grad()

    output = model(source,target)
    output = output[1:].view(-1,output.shape[-1])
    target = target[1:].view(-1)

    loss = criterion(output,target)
    _, predicted = torch.max(output, 1)
    corrects = torch.sum(predicted == target).item()
    total = target.size(0)
    accuracy = corrects / total
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()

    epoch_loss+=loss.item()
    epoch_corrects+=corrects
    epoch_total+=total
  return epoch_loss/len(data_iterator),epoch_corrects/epoch_total

def evaluate(model,data_iterator,criterion):

  model.evaluate()

  epoch_loss = 0
  epoch_total = 0
  epoch_corrects = 0

  for _,(source,target) in enumerate(data_iterator):
    source,target = source.to(device),target.to(device)

    optimizer.zero_grad()

    output = model(source,target)
    output = output[1:].view(-1,output.shape[-1])
    target = target[1:].view(-1)

    _, predicted = torch.max(output, 1)
    corrects = torch.sum(predicted == target).item()
    total = target.size(0)
    accuracy = corrects / total
    loss = criterion(output,target)

    epoch_loss+=loss.item()
    epoch_corrects+=corrects
    epoch_total+=total
  return epoch_loss/len(data_iterator),epoch_corrects/epoch_total
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [52]:
source_vocab_size = len(en_vocab)+5
target_vocab_size = len(fr_vocab)+5
embed_dim = 512
n_heads = 8
num_layers = 6
expansion_factor = 4
max_sentence_length = 50
transformer_model = Transformer(embed_dim,n_heads,expansion_factor,source_vocab_size,target_vocab_size,max_sentence_length,num_layers)
count_parameters(transformer_model)

11137412

In [ ]:
N_EPOCHS = 5
CLIP = 1
loss_f = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = optim.Adam(transformer_model.parameters(),lr = 0.0001,betas = (0.9,0.98),eps = 1e-9)
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
  start_time = time.time()
  train_loss,train_accuracy = train(transformer_model,train_data_token_iter,optimizer,loss_f,CLIP)
  val_loss,val_accuracy = evaluate(transformer_model,valid_data_token_iter,loss_f)
  end_time = time.time()
  epoch_min,epoch_sec = epoch_time(start_time,end_time)

  print(f"Epoch {epoch+1:02} completed | Time : {epoch_min}m and {epoch_sec}s")
  print(f"\tTrain Accuracy : {train_accuracy} | Train Loss : {train_loss}")
  print(f"\tVal Accuracy : {val_accuracy} | Val Loss : {val_loss}")